In [ ]:
%run header.ipynb
%matplotlib inline

In [ ]:
atoms, Ji = FeOnNStevens(
    B20 = -67.72 * meV,
    B40 = -12.90 * meV,
    B44 =   0.16 * meV,
    λso =  -9.41 * meV,
)

wd = "../data"
name = "FeOnNStevens"

Bx = 0.0 * Tesla
By = 0.0 * Tesla
Bz = 2.1 * Tesla


FeOnN = Eigens(wd,
            name = name,
            atoms = atoms,
            Ji = Ji,
            Bx = Bx,
            By = By,
            Bz = Bz,
            maxstates = 1000,
            inmemory = True,
           )

In [ ]:
for i, eV in enumerate((FeOnN.En - FeOnN.En[0])/meV):
    if i == 0: continue
    print("{:>2d}: {:>10.3f} meV".format(i, eV))

In [ ]:
plt.imshow(np.abs(FeOnN.states))

In [ ]:
#electron = SpinAtom(S=1/2, g=2.0, D=0, E=0)
#
#FeOnN = Eigens(wd,
#            name = name,
#            atoms = atoms + [electron, ],
#            Ji = Ji,
#            Bx = Bx,
#            By = By,
#            Bz = Bz,
#            maxstates = 100,
#            inmemory = True,
#           )

In [ ]:
ysyt = YsYt(
    FeOnN, 
    0, 
    u = 0.9,
    use_sparse=False, 
    with_progress=False
)
sss = SteadyStates(
    ysyt,
    u = 0.0,
    η = 0.4, 
    G0 = 0.03 * μS, 
    b0 = 0.6, 
    Gs = 1.4 * μS, 
    bias = np.linspace(-30, 30, 251) * mV,  
    temp = 1.2 * Kelvin,
    algo = "null_space",
) 

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(12, 4))

for E in (FeOnN.En - FeOnN.En[0])/meV:
    if E > 30:
        continue
    for i in [0, 1]:
        ax[i].axvline(E, alpha=0.2)
        if E > 0:
            ax[i].axvline(-E, alpha=0.2)
        
sss.dIdVdf().plot(y=["Current (nA)", "dIdV (nS)"], secondary_y="dIdV (nS)", ax=ax[0])
states = sss.statesdf()
states.plot(ax=ax[1])
ax[1].legend().remove()

fig.suptitle("Fe on N site using Stevens Op.")